### Part Raúl - Elections 1931, 1936, 1939, 1945 & 1950

In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from io import StringIO

# Initialize the WebDriver
driver = webdriver.Chrome()

# Base URL
enlace_base = 'https://infogob.jne.gob.pe/Eleccion'

# List to store tables
elections_1931_1950 = []

# We are selecting the last 5 elections
for k in range(21, 26):
    driver.get(enlace_base)
    time.sleep(2)

    # Select "Tipo de Proceso"
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div').click()
    time.sleep(2)
    # Combine moving down and selecting the option into a single ActionChain
    ActionChains(driver).send_keys(Keys.ARROW_DOWN).send_keys(Keys.ENTER).perform()
    time.sleep(2)  # Add a minimal delay if needed for the page to react

    # Select "Elección"
    dropdown = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div')
    dropdown.click()
    time.sleep(1)  # Short delay to ensure the dropdown is open

    # Move down in the drop-down list k times
    for _ in range(k):
        dropdown.send_keys(Keys.ARROW_DOWN)
        time.sleep(0.5)  # Adjust delay to ensure the dropdown responds

    # Select the currently highlighted option
    dropdown.send_keys(Keys.ENTER)
    time.sleep(2)  # Adjust delay to ensure the selection is made

    # Extract the selected election name
    selected_election_name = dropdown.text

    # View data
    driver.find_element(By.XPATH, '//*[@id="btnVerDatos"]').click()
    time.sleep(10)

    # View results table
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[3]/div[1]/ul/li[2]/a').click()
    time.sleep(2)

    # Extract table data
    table_html = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]').get_attribute('innerHTML')
    table_data = pd.read_html(StringIO(table_html))[0]

    # Add the election name as a new column to the table
    table_data['Elección'] = selected_election_name

    # Keep only the desired columns
    table_data = table_data[['Elección', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]

    elections_1931_1950.append(table_data)

    time.sleep(2)            

    print(f"Data for election number {k}:\n", table_data)
    print("\n" + "=" * 50 + "\n")

driver.quit()

# Concatenate all tables into a single DataFrame
table_1931_1950 = pd.concat(elections_1931_1950, ignore_index=True)

# Save the final table to an Excel file
table_1931_1950.to_excel('Elecciones_Data.xlsx', index=False)

Data for election number 21:
             Elección ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 1950   PARTIDO RESTAURADOR       550779


Data for election number 22:
             Elección        ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 1945  FRENTE DEMOCRATICO NACIONAL       305590
1  PRESIDENCIAL 1945         UNION REVOLUCIONARIA       150720


Data for election number 23:
             Elección   ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 1939  CONCENTRACION NACIONAL       262971
1  PRESIDENCIAL 1939       FRENTE PATRIOTICO        76222


Data for election number 24:
             Elección     ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 1936  PARTIDO SOCIAL DEMOCRATA        77042
1  PRESIDENCIAL 1936      UNION REVOLUCIONARIA        57838
2  PRESIDENCIAL 1936           FRENTE NACIONAL        46773
3  PRESIDENCIAL 1936       PARTIDO REPUBLICANO        30803


Data for election number 25:
             Elección       ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PR

In [3]:
## Finally, we can see the required table
table_1931_1950

,Elección,ORGANIZACIÓN POLÍTICA,TOTAL VOTOS
0,PRESIDENCIAL 1950,PARTIDO RESTAURADOR,550779
1,PRESIDENCIAL 1945,FRENTE DEMOCRATICO NACIONAL,305590
2,PRESIDENCIAL 1945,UNION REVOLUCIONARIA,150720
3,PRESIDENCIAL 1939,CONCENTRACION NACIONAL,262971
4,PRESIDENCIAL 1939,FRENTE PATRIOTICO,76222
5,PRESIDENCIAL 1936,PARTIDO SOCIAL DEMOCRATA,77042
6,PRESIDENCIAL 1936,UNION REVOLUCIONARIA,57838
7,PRESIDENCIAL 1936,FRENTE NACIONAL,46773
8,PRESIDENCIAL 1936,PARTIDO REPUBLICANO,30803
9,PRESIDENCIAL 1931,UNION REVOLUCIONARIA,152149


### Part Antonio Soto - Elections 2011-2021

In [69]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from io import StringIO

# Initialize the WebDriver
driver = webdriver.Chrome()

# Base URL
base_url = 'https://infogob.jne.gob.pe/Eleccion'

# Load the existing Excel file
try:
    existing_data = pd.read_excel('Elecciones_Data.xlsx')
except FileNotFoundError:
    print("The file 'Elecciones_Data.xlsx' does not exist, a new one will be created.")
    existing_data = pd.DataFrame()  # Create an empty DataFrame if the file does not exist

# List to store new tables
elections_data = []

# List of elections to process
elections_list = [
    "PRESIDENCIAL 2021 - 2DA VUELTA",
    "PRESIDENCIAL 2021",
    "PRESIDENCIAL 2016 - 2DA VUELTA",
    "PRESIDENCIAL 2016",
    "PRESIDENCIAL 2011"
]

# Navigate and select the elections
for election in elections_list:
    driver.get(base_url)
    time.sleep(2)

    # Select "Tipo de Proceso"
    tipo_proceso_dropdown = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[1]/div')
    tipo_proceso_dropdown.click()
    time.sleep(2)

    # Use JavaScript to select the option if necessary
    driver.execute_script("arguments[0].click();", tipo_proceso_dropdown)
    ActionChains(driver).send_keys("ELECCIONES PRESIDENCIALES").send_keys(Keys.ENTER).perform()
    time.sleep(2)

    # Manually open the election dropdown
    election_dropdown = driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[2]/div[2]/div[2]/div')
    election_dropdown.click()
    time.sleep(1)

    # Find and click the specific election option
    option_xpath = f"//div[@class='item' and text()='{election}']"
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, option_xpath))
    ).click()
    time.sleep(2)

    # Click "Ver Datos de la Elección"
    boton_ver_datos = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="btnVerDatos"]'))
    )
    boton_ver_datos.click()
    time.sleep(10)

    # View results table
    driver.find_element(By.XPATH, '//*[@id="section"]/div[2]/div[3]/div[1]/ul/li[2]/a').click()
    time.sleep(2)

    # Extract table data
    table_html = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[3]/div/div/div/div[1]/div[2]/div[2]').get_attribute('innerHTML')
    table_data = pd.read_html(StringIO(table_html))[0]

    # Add the election name as a new column to the table
    table_data['Elección'] = election

    # Keep only the desired columns
    table_data = table_data[['Elección', 'ORGANIZACIÓN POLÍTICA', 'TOTAL VOTOS']]

    elections_data.append(table_data)

    time.sleep(2)

    print(f"Data for {election}:\n", table_data)
    print("\n" + "=" * 50 + "\n")

driver.quit()

# Combine the new tables into a single DataFrame
new_data = pd.concat(elections_data, ignore_index=True)

# Combine existing data with new data
final_data = pd.concat([existing_data, new_data], ignore_index=True)

# Save the combined table to the Excel file
final_data.to_excel('Elecciones_Data.xlsx', index=False)

Data for PRESIDENCIAL 2021 - 2DA VUELTA:
                          Elección                 ORGANIZACIÓN POLÍTICA  \
0  PRESIDENCIAL 2021 - 2DA VUELTA  PARTIDO POLITICO NACIONAL PERU LIBRE   
1  PRESIDENCIAL 2021 - 2DA VUELTA                        FUERZA POPULAR   
2  PRESIDENCIAL 2021 - 2DA VUELTA                       VOTOS EN BLANCO   
3  PRESIDENCIAL 2021 - 2DA VUELTA                           VOTOS NULOS   

   TOTAL VOTOS  
0      8836380  
1      8792117  
2       121489  
3      1106816  


Data for PRESIDENCIAL 2021:
             Elección                        ORGANIZACIÓN POLÍTICA  TOTAL VOTOS
0  PRESIDENCIAL 2021         PARTIDO POLITICO NACIONAL PERU LIBRE      2724752
1  PRESIDENCIAL 2021                               FUERZA POPULAR      1930762
2  PRESIDENCIAL 2021                           RENOVACION POPULAR      1692279
3  PRESIDENCIAL 2021  AVANZA PAIS - PARTIDO DE INTEGRACION SOCIAL      1674201
4  PRESIDENCIAL 2021                               ACCION POPULAR      

In [71]:
## Finally, we can see the required table
final_data

,Elección,ORGANIZACIÓN POLÍTICA,TOTAL VOTOS
0,PRESIDENCIAL 1950,PARTIDO RESTAURADOR,550779
1,PRESIDENCIAL 1945,FRENTE DEMOCRATICO NACIONAL,305590
2,PRESIDENCIAL 1945,UNION REVOLUCIONARIA,150720
3,PRESIDENCIAL 1939,CONCENTRACION NACIONAL,262971
4,PRESIDENCIAL 1939,FRENTE PATRIOTICO,76222
...,...,...,...
66,PRESIDENCIAL 2011,FONAVISTAS DEL PERU,37011
67,PRESIDENCIAL 2011,DESPERTAR NACIONAL,21574
68,PRESIDENCIAL 2011,PARTIDO POLITICO ADELANTE,17301
69,PRESIDENCIAL 2011,FUERZA NACIONAL,16831
